# Value Iteration for Frozen Lake Problem

## 1. Import necessary libraries

In [1]:
import gym
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
import time
%matplotlib inline

## 2. Initialize our gym environment

In [3]:
env = gym.make('FrozenLake-v0')
env = env.unwrapped
print(env)

<FrozenLakeEnv<FrozenLake-v0>>


## 3. See the number of states and actions in FrozenLakeEnv

In [4]:
print("number of states: ", env.observation_space.n)
print("number of actions: ", env.action_space.n)

number of states:  16
number of actions:  4


## 4. Let us see how the environment looks like

In [5]:
env.render() # environment가 어떻게 생겼는지 확인


SFFF
FHFH
FFFH
HFFG


* 16개 grid, starting point에서 출발하여 hole에 빠지지 않고 goal로 가는 것이 목표!

## 5. Value Iteration 함수 정의

In [8]:
def value_iteration(env, gamma = 1.0):
    # initialize value table with zeros
    value_table = np.zeros(env.observation_space.n)
    
    # set number of iterations and threshold
    no_of_iterations = 100000
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        # On each iteration, copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table)
        
        # Now we calculate Q Value for each actions in the state
        # and update the value of a state with maximum Q value
        
        ################################################################################################
        # Calculating Q value
        ################################################################################################
        for state in range(env.observation_space.n):
            # Q value for current state, Q_value = [Q(s,a1), Q(s,a2), Q(s,a3), Q(s,a4)]
            Q_value = []
            for action in range(env.action_space.n):
                # Q value for each action, next_states_rewards = [rewards(s,a1,s2), rewards(s,a1,s3), ...]
                next_states_rewards = []
                for next_sr in env.P[state][action]:
                    trans_prob, next_state, reward_prob, _ = next_sr
                    next_states_rewards.append((trans_prob * (reward_prob + gamma * updated_value_table[next_state])))
                
                Q_value.append(np.sum(next_states_rewards))
            
            value_table[state] = max(Q_value)
        
        # we will check whether we have reached the convergence i.e. whether the difference between our
        # value table and updated value table is very small. But how do we know it is very small? We set
        # some threshold and then we will see if the difference is less than our threshold, if it is
        # less, we break the loop and return the value function as optimal value function.
        
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
            print('Value-iteration converged at iteration # %d.' %(i+1))
            break
    
    return value_table

## 6. Extract policy 함수 정의

In [9]:
def extract_policy(value_table, gamma=1.0):
    # initialize the policy with zeros
    policy = np.zeros(env.observation_space.n, np.int32) # size = state 개수
    
    for state in range(env.observation_space.n):
        # initialize the Q table for a state
        Q_table = np.zeros(env.action_space.n)
        
        # compute Q value for all actions in the state
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]:
                trans_prob, next_state, reward_prob, _ = next_sr
                Q_table[action] += (trans_prob * (reward_prob + gamma * value_table[next_state]))
        
        # select the action which has maximum Q value as an optimal action of the state
        policy[state] = np.argmax(Q_table) # argmax : Q value 중 가장 큰 값의 index 반환
    
    return policy

## 7. Rendering for visualization

In [10]:
def play(env, optimal_policy, max_step=1000):
    state = 0
    for i in range(max_step):
        env.render()
        time.sleep(1)
        display.clear_output(wait=True)
        display.display(plt.gcf())
        state, _, done, _ = env.step(optimal_policy[state])
        
        if done:
            env.render()
            break

# Value Iteration 실행

## 1. First, we compute the optimal value function

In [11]:
optimal_value_function = value_iteration(env=env, gamma=1.0)

Value-iteration converged at iteration # 1373.


## 2. and we derive the optimal policy from the optimal value function

In [12]:
optimal_policy = extract_policy(optimal_value_function, gamma=1.0)

In [13]:
print(optimal_policy)

[0 3 3 3 0 0 0 0 3 1 0 0 0 2 1 0]


In [14]:
play(env, optimal_policy)

  (Down)
SFFF
FHFH
FFFH
HFFG


# Policy Iteration for Frozen Lake Problem

## 1. Value function 계산 함수

In [15]:
def compute_value_function(policy, gamma=1.0):
    # initialize value table with zeros
    value_table = np.zeros(env.nS)
    
    # set the threshold
    threshold = 1e-10
    
    while True:
        # copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table)
        
        # for each state in the environment, select the action according to the policy and compute the value table
        for state in range(env.nS):
            action = policy[state]
            
            # build the value table with the selected action
            value_table[state] = sum([trans_prob * (reward_prob + gamma * updated_value_table[next_state])
                                     for trans_prob, next_state, reward_prob, _ in env.P[state][action]])
        
        if (np.sum((np.fabs(updated_value_table - value_table))) <= threshold):
            break
    
    return value_table

* random policy -> evaluation(with value function) -> improvement -> ...

## 2. Extract Policy 함수 - 위와 동일

## 3. Policy Iteration 함수

In [16]:
def policy_iteration(env, gamma=1.0):
    # Initialize policy with zeros
    old_policy = np.zeros(env.observation_space.n, np.int32)
    no_of_iterations = 200000
    
    for i in range(no_of_iterations):
        # compute the value function
        new_value_function = compute_value_function(old_policy, gamma)
        
        # Extract new policy from the computed value function
        new_policy = extract_policy(new_value_function, gamma)
        
        # Then we check whether we have reached convergence i.e. whether we found the optimal policy by
        # comparing old_policy and new policy if it same we will break the iteration
        # else we update old_policy with new_policy
        
        if (np.all(old_policy == new_policy)):
            print('Policy-Iteration converged at step %d.' %(i+1))
            break
        old_policy = new_policy
    
    return new_policy

* 앞의 두 함수들을 반복적으로 수행

## 4. Rendering for visualization

In [20]:
def play(env, optimal_policy, max_step=1000):
    state = 0
    for i in range(max_step):
        env.render()
        time.sleep(1)
        display.clear_output(wait=True)
        display.display(plt.gcf())
        state, _, done, _ = env.step(optimal_policy[state])
        
        if done:
            env.render()
            break

# Policy Iteration 실행

In [18]:
optimal_policy = policy_iteration(env)
print(optimal_policy)

Policy-Iteration converged at step 7.
[0 3 3 3 0 0 0 0 3 1 0 0 0 2 1 0]


In [21]:
play(env, optimal_policy)

  (Left)
SFFF
FHFH
FFFH
HFFG
